In [1]:
pip install --upgrade MetaTrader5

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5
from zoneinfo import ZoneInfo
import pytz
import time
from IPython.display import clear_output
from pandas import Timestamp

In [3]:
# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("Initialization failed")
    quit()

DEFINING FUNCTIONS

In [4]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame_input):
    rates_frame_input['time'] = pd.to_datetime(rates_frame_input['time'], unit='s')
    rates_frame_input.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame_input["time"] = pd.to_datetime(rates_frame_input["time"])
    return rates_frame

def append_new_data(rates_frame, rates_frame_to_append):
    rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
    return rates_frame

def download_recent_OHLCdata():
    symbol = "EURUSD"
    timeframe = mt5.TIMEFRAME_M1
    
    #GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
    utc_from = datetime.now() - timedelta(minutes=3) + timedelta(hours=2)
    utc_to = datetime.now() + timedelta(hours=2) - timedelta(minutes=1)
    startdate=utc_from.strftime('%Y%m%d')
    enddate=utc_to.strftime('%Y%m%d')
    
    # Get the OHLC data FOR TODAY AND YESTERDAY
    rates_5m = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    # Check if the data is downloaded successfully
    if rates is None:
        print("No data available, error code =", mt5.last_error())
    else:
        # Convert the data to a pandas DataFrame
        rates_frame_to_append = pd.DataFrame(rates_5m)
        df_convert_hr(rates_frame_to_append)
        return rates_frame_to_append

def download_and_display_updated_values():
    global rates_frame
    rates_frame_to_append = download_recent_OHLCdata()
    rates_frame = append_new_data(rates_frame, rates_frame_to_append)
    print("  ")
    print(f"{symbol} OHLC prices from the last 10 minutes :")
    print("____________________________________________________________________________")
    print(rates_frame.tail(10))
    return

In [5]:
#Detect the entry drills
def check_buy_entry_drill():
    if buy_tp == "-":
        return
        
    global rates_frame, Last_110
    
    #check if pattern occured during london session:
    ldn_session = 10 < LowestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during ny session
    ny_session = 15 < LowestPoint["time"].to_pydatetime().hour < 19
    #check if the drill appreared during pre-session times
    pre_ldn_session = 9 < LowestPoint["time"].to_pydatetime().hour <= 10
    pre_ny_session = 14 <= LowestPoint["time"].to_pydatetime().hour <= 15

    if ldn_session and (ldn_low == LowestPoint["low"]):
        print("Price is at OTE!")
        print("Current session: London")
        print(f"London session low: {ldn_low}")
        print(f"Lowest point from the last 110 minutes: {LowestPoint["low"]}")
        #insert code here
        return
    elif ny_session and (ny_low == LowestPoint["low"]):
        print("Price is at OTE!")
        print("Current session: NY")
        print(f"NY session low: {ny_low}")
        print(f"Lowest point from the last 110 minutes: {LowestPoint["low"]}")
        #insert code here
        return
    elif (pre_ldn_session or pre_ny_session):
        print("Current session: Pre-London or Pre-NY")
        print(f"Lowest point from the last 110 minutes: {LowestPoint["low"]}")
        #insert code here
        return
    else:
        print("Price is not at the OTE of the session!")
        return

def check_sell_entry_drill():
    if sell_tp == "-":
        return
        
    global rates_frame, Last_110
    
    #check if pattern occured during london session:
    ldn_session = 10 < HighestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during ny session
    ny_session = 15 < HighestPoint["time"].to_pydatetime().hour < 19
    #check if the drill appreared during pre-session times
    pre_ldn_session = 9 < HighestPoint["time"].to_pydatetime().hour <= 10
    pre_ny_session = 14 <= HighestPoint["time"].to_pydatetime().hour <= 15
    
    if ldn_session and (ldn_high == HighestPoint["high"]):
        print("Price is at OTE!")
        print("Current session: London")
        print(f"London session high: {ldn_high}")
        print(f"Highest point from the last 110 minutes: {HighestPoint["high"]}")
        #insert code here
        return
    elif ny_session and (ny_high == HighestPoint["high"]):
        print("Price is at OTE!")
        print("Current session: NY")
        print(f"NY session high: {ny_high}")
        print(f"Highest point from the last 110 minutes: {HighestPoint["high"]}")
        #insert code here
        return
    elif (pre_ldn_session or pre_ny_session):
        print("Current session: Pre-London or Pre-NY")
        print(f"Highest point from the last 110 minutes: {HighestPoint["high"]}")
        #insert code here
        return
    else:
        print("Price is not at the OTE of the session!")
        return

DOWNLOAD THE OHLC DATA FROM THE LAST 7 DAYS

In [6]:
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
utc_from = datetime.now() - timedelta(days=1) + timedelta(hours=2)
utc_to = datetime.now() - timedelta(minutes=1) + timedelta(hours=2)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data FOR TODAY AND YESTERDAY
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    df_convert_hr(rates_frame)


In [7]:
rates_frame

,time,open,high,low,close
0,2025-01-09 17:34:00,1.03020,1.03026,1.03019,1.03026
1,2025-01-09 17:35:00,1.03026,1.03028,1.03018,1.03026
2,2025-01-09 17:36:00,1.03026,1.03026,1.03006,1.03007
3,2025-01-09 17:37:00,1.03008,1.03009,1.03003,1.03005
4,2025-01-09 17:38:00,1.03005,1.03009,1.03001,1.03001
...,...,...,...,...,...
1434,2025-01-10 17:28:00,1.02671,1.02692,1.02663,1.02664
1435,2025-01-10 17:29:00,1.02662,1.02677,1.02661,1.02675
1436,2025-01-10 17:30:00,1.02675,1.02675,1.02631,1.02641
1437,2025-01-10 17:31:00,1.02637,1.02642,1.02612,1.02617


GRAB USER INPUT FOR THE TAKE PROFIT LEVELS FOR THE BUYS AND SELLS

In [8]:
current_price = float(rates_frame["close"].tail(1).iloc[0])
print(f"The current price of {symbol} is about: {current_price}.")
while True:
    sell_tp = input("Enter the sell TP: ").strip()
    try: 
        # Check for a dash
        if sell_tp == "-":
            print("You are only looking to buy.")
            BEARISH_BIAS = False
            break
    
        sell_tp = float(sell_tp)
        if sell_tp >= current_price:
            print(f"Enter a price that is less than the current price of {symbol}.")
            continue
        value = float(sell_tp)
        print(f"You entered a TP level: {value}")
        BEARISH_BIAS = True
        break
    except ValueError:
        print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
        continue

while True:
    buy_tp = input("Enter the buy TP: ").strip()
    try:
        if buy_tp == "-":
            print("You are only looking to sell.")
            BULLISH_BIAS = False
            break
        
        buy_tp = float(buy_tp) 
        if buy_tp <= current_price:
            print(f"Enter a price that is more than the current price of {symbol}.")
            continue
            
        value = float(buy_tp)
        print(f"You entered a TP level: {value}")
        BULLISH_BIAS = True
        break
    except ValueError:
        print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
        continue

The current price of EURUSD is about: 1.02612.


Enter the sell TP:  0


You entered a TP level: 0.0


Enter the buy TP:  2


You entered a TP level: 2.0


CREATING A RISK MANAGEMENT STRATEGY THAT TRADES ONLY A MAXIMUM OF 4 TRADES PER DAY

In [ ]:
#number of trades for today
N = 0
date_today = date.today()

while N < 4:
    if N > 4:
        break
    else:
        clear_output(wait=True)
        #HAVE THE DATAFRAME FOR THE CURRENT LONDON SESSION and LOOK FOR THE LONDON HIGH AND LOW:
        ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 10) & (rates_frame['time'].dt.hour < 14)]
        ldn_high = ldn_rates_frame["high"].max()
        ldn_low = ldn_rates_frame["low"].min()
        print("__________________________________")
        print(f'London high: {ldn_high}')
        print(f'London low: {ldn_low}')
        
        #HAVE THE DATAFRAME FOR THE CURRENT NEW YORK SESSION and LOOK FOR THE NY HIGH AND LOW: 
        ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 15) & (rates_frame['time'].dt.hour < 19)]
        ny_high = ny_rates_frame["high"].max()
        ny_low = ny_rates_frame["low"].min()
        print(f'NY high: {ny_high}')
        print(f'NY low: {ny_low}')
        print("__________________________________")
        
        #Determine the max and min value of the highs and lows for the last 35 mins
        Last_110 = rates_frame.tail(110)
        MaxHighPrice = Last_110["high"].max()
        MinLowPrice = Last_110["low"].min()

        #Determine the if the last 80 mins is in a downtrend or uptrend
        HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
        LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]

        #Based on the trend (and the conditions inside) decide whether to buy, sell, or wait.
        if (HighestPoint["time"] > LowestPoint["time"]) & BEARISH_BIAS:
            print("Looking to sell...")
            check_sell_entry_drill()
            download_and_display_updated_values()
            time.sleep(60)
            continue
        elif (HighestPoint["time"] < LowestPoint["time"]) & BULLISH_BIAS:
            print("Looking to buy...")
            check_buy_entry_drill()
            download_and_display_updated_values()
            time.sleep(60)
            continue
        else:
            print("Waiting for an opportunity...")
            download_and_display_updated_values()
            time.sleep(60)
            continue

__________________________________
London high: 1.03117
London low: 1.02926
NY high: 1.03117
NY low: 1.02128
__________________________________
Looking to buy...
Price is not at the OTE of the session!
  
EURUSD OHLC prices from the last 10 minutes :
____________________________________________________________________________
                    time     open     high      low    close
1566 2025-01-10 19:40:00  1.02458  1.02469  1.02453  1.02464
1567 2025-01-10 19:41:00  1.02464  1.02483  1.02454  1.02476
1568 2025-01-10 19:42:00  1.02475  1.02480  1.02469  1.02479
1569 2025-01-10 19:43:00  1.02477  1.02505  1.02466  1.02497
1570 2025-01-10 19:44:00  1.02499  1.02503  1.02482  1.02485
1571 2025-01-10 19:45:00  1.02485  1.02493  1.02467  1.02470
1572 2025-01-10 19:46:00  1.02470  1.02480  1.02468  1.02469
1573 2025-01-10 19:47:00  1.02469  1.02483  1.02468  1.02480
1574 2025-01-10 19:48:00  1.02479  1.02489  1.02468  1.02475
1575 2025-01-10 19:49:00  1.02474  1.02474  1.02461  1.02464
